In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import plotly.express as px
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
# Import libraries and set desired options
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import hstack
# !pip install eli5
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display_html


In [ ]:
test = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
train = pd.read_csv('../input/GiveMeSomeCredit/cs-training.csv')

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
logit = LogisticRegression(C=1, random_state=17, solver='liblinear')

In [ ]:
%%time

cv_scores1 = cross_val_score(logit, train, train.SeriousDlqin2yrs, cv=5, 
                            scoring='roc_auc', n_jobs=4) # hangs with n_jobs > 1, and locally this runs much faster

In [ ]:
cv_scores1, cv_scores1.mean()

In [ ]:
logit.fit(train, train.SeriousDlqin2yrs)

In [ ]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
logit_test_pred = logit.predict_proba(test)[:, 1]
write_to_submission_file(logit_test_pred, 'subm1.csv') # 0.91807

In [ ]:
subm = pd.read_csv('./subm1.csv')
subm

In [ ]:
def train_and_predict(model, X_train, y_train, X_test, site_feature_names=train.columns, 
                      new_feature_names=None, cv=5, scoring='roc_auc',
                      top_n_features_to_show=30, submission_file_name='submission.csv'):
    
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring=scoring, n_jobs=4)
    print('CV scores', cv_scores)
    print('CV mean: {}, CV std: {}'.format(cv_scores.mean(), cv_scores.std()))
    model.fit(X_train, y_train)
    
    if new_feature_names:
        all_feature_names = site_feature_names + new_feature_names 
    else: 
        all_feature_names = site_feature_names

    
    if new_feature_names:
        print('New feature weights:')
    
        print(pd.DataFrame({'feature': new_feature_names, 
                        'coef': model.coef_.flatten()[-len(new_feature_names):]}))
    
    test_pred = model.predict_proba(X_test)[:, 1]
    write_to_submission_file(test_pred, submission_file_name) 
    
    return cv_scores

In [ ]:
cv_scores1 = train_and_predict(model=logit, X_train=train, y_train=train.SeriousDlqin2yrs, 
                  X_test=test, site_feature_names=train.columns,              
                  cv=5, submission_file_name='subm1.csv')

# Start

In [ ]:
train = train[['SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents']]
test = test[['SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents']]

In [ ]:
test['MonthlyIncome'] = test['MonthlyIncome'].fillna(0.0)
train['MonthlyIncome'] = train['MonthlyIncome'].fillna(0.0)
test['NumberOfDependents'] = test['NumberOfDependents'].fillna(test['NumberOfDependents'].median())
train['NumberOfDependents'] = train['NumberOfDependents'].fillna(train['NumberOfDependents'].median())

In [ ]:
independent_columns_names = [x for x in train if x != 'SeriousDlqin2yrs']
independent_columns_names

In [ ]:
table = train
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
lr = LogisticRegression(C=0.001, random_state=5, class_weight='balanced')
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

In [ ]:
# BEFORE WAS: logit = LogisticRegression(C=1, random_state=17, solver='liblinear')
logit = LogisticRegression(C=0.001, random_state=5, class_weight='balanced', solver='liblinear')

In [ ]:
%%time

cv_scores1 = cross_val_score(logit, train[['NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse']], train.SeriousDlqin2yrs, cv=5, 
                            scoring='roc_auc', n_jobs=4) # hangs with n_jobs > 1, and locally this runs much faster

In [ ]:
cv_scores1, cv_scores1.mean()

In [ ]:
logit.fit(train, train.SeriousDlqin2yrs)

In [ ]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
test

In [ ]:

logit_test_pred = logit.predict_proba(test)[:, 0]
write_to_submission_file(logit_test_pred, 'subm3.csv') # 0.91807

In [ ]:
subm = pd.read_csv('./subm3.csv')
subm

In [ ]:
def train_and_predict(model, X_train, y_train, X_test, site_feature_names=train.columns, 
                      new_feature_names=None, cv=5, scoring='roc_auc',
                      top_n_features_to_show=30, submission_file_name='submission.csv'):
    
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring=scoring, n_jobs=4)
    print('CV scores', cv_scores)
    print('CV mean: {}, CV std: {}'.format(cv_scores.mean(), cv_scores.std()))
    model.fit(X_train, y_train)
    
    if new_feature_names:
        all_feature_names = site_feature_names + new_feature_names 
    else: 
        all_feature_names = site_feature_names

    
    if new_feature_names:
        print('New feature weights:')
    
        print(pd.DataFrame({'feature': new_feature_names, 
                        'coef': model.coef_.flatten()[-len(new_feature_names):]}))
    
    test_pred = model.predict_proba(X_test)[:, 1]
    write_to_submission_file(test_pred, submission_file_name) 
    
    return cv_scores

In [ ]:
cv_scores1 = train_and_predict(model=logit, X_train=train, y_train=train.SeriousDlqin2yrs, 
                  X_test=test, site_feature_names=train.columns,              
                  cv=5, submission_file_name='subm1.csv')

In [ ]:
# Let's load the packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

In [ ]:
X = df
y = df.SeriousDlqin2yrs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

In [ ]:
df.columns

In [ ]:
plt.barh(df[['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents']], rf.feature_importances_)

In [ ]:
sorted_idx = rf.feature_importances_.argsort()
plt.barh(df.columns[sorted_idx], rf.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")